In [4]:
import gmsh

In [34]:
gmsh.initialize()

gmsh.model.add("Student's room")

# Параметры комнаты
room_width = 3.8
room_height = 3.1
room_length = 5.6

# Параметры окна
window_width = 1.6
window_length = 1.5
window_offset = 1.4     # отступ окна от пола

# Параметры батареи
heater_width = 1.5
heater_height = 1
heater_depth = -0.14    # толщина батареи
heater_offset = 0.2   # расстояние от пола до нижней части батареи

room = gmsh.model.occ.addBox(0, 0, 0, room_width, room_length, room_height)
x_window = (room_width - window_width) / 2
y_window = room_length
z_window = window_offset
window = gmsh.model.occ.addBox(x_window, y_window, z_window, window_width, 0.1, window_height)


x_heater = x_window + (window_width - heater_width) / 2  # центрирование батареи под окном
y_heater = y_window
z_heater = heater_offset
heater = gmsh.model.occ.addBox(x_heater, y_heater, z_heater, heater_width, heater_depth, heater_height)

gmsh.model.occ.cut([(3, room)], [(3, window)], removeTool=True)
air_volume = gmsh.model.occ.cut([(3, room)], [(3, heater)], removeTool=True)
gmsh.model.occ.synchronize()

# Уменьшение размера сетки на радиаторе
# Получаем все поверхности радиатора
heater_surfaces = gmsh.model.getEntitiesInBoundingBox(x_heater, y_heater, z_heater,
                                                      x_heater + heater_width, y_heater + heater_depth, z_heater + heater_height, 2)

# Применение локального размера сетки на поверхности радиатора
for surface in heater_surfaces:
    gmsh.model.mesh.setSize([surface], 0.05)  # Установите желаемый размер сетки, например, 0.05
gmsh.model.mesh.generate(3)

gmsh.write("room_with_window_and_heater.vtk")

gmsh.finalize()